In [1]:
import doctest
import random
import re
import time
import pickle
import xml.etree.ElementTree as ET
import json
import os
import sys

In [2]:
def p_equal(p1, p2):
    if len(p1) > 0 and p1[-1] == '$':
        p1 = p1[:-1]
    if len(p2) > 0 and p2[-1] == '$':
        p2 = p2[:-1]
    return p1 == p2

def p_match(p1, p2):
    if p1[-1:] == ['$']:
        if len(p1) != len(p2) or p2[-1:] != ['$']:
            return False
    else:
        if len(p1) > len(p2):
            return False
    return p1 == p2[:len(p1)]

def p_diff(p1, p2):
    for i in range(len(p1)):
        if i >= len(p2) or p1[i] != p2[i]:
            return p1[i:]
    return []

In [26]:
p_diff(['A','B','$'],['ROOT NODE'])

['A', 'B', '$']

In [3]:
SIMILARITY_THRESHOLD = 5
ADD_LINK_TIME = 10000
DISCRIMINATION_TIME = 10000
FAMILIARISATION_TIME = 2000

In [4]:
class Stm:
    class Item:   #create an item container with head and pointer to next container                
        def __init__(self, data):    
            self.next = None
            self.data = data

    def __init__(self, maxlen): #STM constructor
        self.maxlen = maxlen
        self.size = 0
        self.head = None
        self.tail = None

    def push(self, data):
        self.remove(data)
        e = Stm.Item(data)
        if self.size == 0:
            self.head = self.tail = e
        else:
            check = self.head.data.image.copy()
            k = 0
            # print('check similarity', check, data.image)
            matches = []
            for i in data.image:
                if i in check:
                    k += 1
                    matches.append((data.idx, i, check))
                    if k == SIMILARITY_THRESHOLD:
                        self.head.data.similarity_links.add(data)
                        data.similarity_links.add(self.head.data)
#                         print('------------------------')
#                         for idx, i, check in matches:
#                             print(data.idx, 'FOUND', i, 'in', check)
#                         print('------------------------')
                        break
                    check.remove(i)
            self.tail.next = e
            self.tail = e
        self.size += 1
        if self.size > self.maxlen:
            self.pop()

    def pop(self):
        if self.size == 0:
            return None
        else:
            e = self.head.data
            self.head = self.head.next
            self.size -= 1
            return e

    def remove(self, value):
        if self.size == 0:
            return
        if self.head.data == value:
            self.head = self.head.next
            self.size -= 1
        else:
            p = self.head
            e = self.head.next
            while e:
                if e.data == value:
                    p.next = e.next
                    self.size -= 1
                    if e == self.tail:
                        self.tail = p
                    break
                p = p.next
                e = e.next

    class Iterator:
        def __init__(self, e):
            self.e = e

        def __next__(self):
            if self.e:
                e = self.e
                self.e = e.next
                return e.data
            raise StopIteration()

    def __iter__(self):
        return Stm.Iterator(self.head)

In [5]:
class Link:
    def __init__(self, test, child):
        self.test = test
        self.child = child
    
    def passes(self, pattern):
        return p_match(self.test, pattern)

In [6]:
class Node:
    def __init__(self, nodes, contents, image, children):
        self.contents = contents
        self.image = image
        if len(self.image) > 1 and self.image[-1] == '$':
            self.image.pop()
        self.children = children
        self.idx = len(nodes)
        nodes.append(self)
        self.label = None
        self.similarity_links = set()
#         print(Node.__dict__)

In [7]:
class Modality:
    def __init__(self, chrest, mod_name, stm_size=3): #default stm_size=3
        self.chrest = chrest
        self.mod_name = mod_name
        self.stm = Stm(stm_size)
        self.net = Node(nodes=chrest.nodes, contents=['ROOT NODE'], image=[], children=[])

    def recognise(self, pattern):
        current_node = self.net
        children = current_node.children
        sorted_pattern = pattern
        next_link = 0
        
        while next_link < len(children):
            link = children[next_link] #one of the Link objects in the children []
            if link.passes(sorted_pattern):
                current_node = link.child
                children = current_node.children
                next_link = 0
                sorted_pattern = p_diff(sorted_pattern, link.test)
            else:
                next_link += 1
        self.stm.push(current_node)
        return current_node
    
    def recognise_and_learn(self, pattern):
        current_node = self.recognise(pattern)
        if current_node.image != pattern:
            if current_node == self.net or \
               not p_match(current_node.image, pattern) or \
               current_node.image[-1:] == ['$']:
                current_node = self.discriminate(current_node, pattern)
            else:
                current_node = self.familiarise(current_node, pattern)
        self.stm.push(current_node)
        return current_node
    
    def get_first(self, pattern):
        fst = pattern[:1]
        if len(fst) == 0 or fst == ['$']:
            return []
        else:
            return fst
    
    def familiarise(self, node, pattern):
        new_info = self.get_first(p_diff(pattern, node.image))
        if len(new_info) == 0:
            return node
        
        retrieved_chunk = self.recognise(new_info)
        if retrieved_chunk == self.net:
            return self.learn_primitive(new_info)
        
        if node.image[:-1] == ['$']:
            node.image.pop()
        node.image += new_info
        self.chrest.clock += FAMILIARISATION_TIME
        return node
    
    def discriminate(self, node, pattern):
        new_info = p_diff(pattern, node.contents)
        
        if len(new_info) == 0 or new_info == ['$']:
            new_info = ['$']
            if self.recognise(new_info).contents == new_info:
                return self.add_test(node, new_info)
            else:
                child = Node(nodes=self.chrest.nodes, contents=new_info, image=new_info, children=[])
                self.net.children.append(Link(new_info, child))
                return child
        
        chunk = self.recognise(new_info)
        if chunk == self.net:
            return self.learn_primitive(self.get_first(new_info) or ['$'])
        elif p_match(chunk.contents, new_info):
            return self.add_test(node, chunk.contents.copy())
        else:
            return self.add_test(node, self.get_first(new_info))
            

    def add_test(self, node, pattern):
        for child in node.children:
            if child.test == pattern:
                return node
        if node == self.net:
            pat = pattern.copy()
        else:
            pat = node.contents + pattern
        child = Node(nodes=self.chrest.nodes, contents=pat, image=pat.copy(), children=[])
        node.children.append(Link(pattern, child))
        self.chrest.clock += DISCRIMINATION_TIME
        return child
    
    def learn_primitive(self, pattern):
        assert len(pattern) == 1
        child = Node(nodes=self.chrest.nodes, contents=pattern, image=[], children=[])
        self.net.children.append(Link(pattern, child))
        self.chrest.clock += DISCRIMINATION_TIME
        return child
    
    def print_tree(self, node=None, parent=[], level=0):
        if not node:
            node = self.net
        indent = '-------' * level
        contents = '< ' + ' '.join(p_diff(node.contents, parent)) + ' >'
        text = 'Node: ' + str(node.idx)
        image = '< '+ ' '.join(node.image) + ' >'
        
        string = [indent + contents, text, image]
        if node.label is not None:
            string.append('(' + str(node.label) + ')')
        if node.similarity_links:
            string.append({i.idx for i in node.similarity_links})
        print(*string)

        for child in node.children:
            self.print_tree(child.child, node.contents, level + 1)

    def print_stm(self):
        print('STM: ', end='')
        for node in self.stm:
            print(node.idx, '<', *node.image, '>', end=', ')
        print()


In [21]:
class Chrest:
    def __init__(self):
        self.nodes = []
        self.visual = Modality(self, 'visual')
        self.verbal = Modality(self, 'verbal')
        self.action = Modality(self, 'action')
        self.modalities = {
            'visual': self.visual,
            'verbal': self.verbal,
            'action': self.action,
        }
        self.clock = 0

# Basic Tests

In [24]:
c= Chrest()
c.verbal.print_tree()
c.verbal.net.contents


< ROOT NODE > Node: 1 <  >


['ROOT NODE']

In [28]:
c.verbal.recognise_and_learn(list('AB$'))
c.verbal.print_tree()


< ROOT NODE > Node: 1 <  >
-------< A > Node: 3 <  >


In [9]:
chrest1 = Chrest()   #reset LTM to blank
chrest1.verbal.print_tree()
chrest1.visual.print_tree()

< ROOT NODE > Node: 1 <  >
< ROOT NODE > Node: 0 <  >


In [23]:
chrest1.verbal.recognise_and_learn(list('AB$'))        #rerun few times to learn fully
chrest1.verbal.print_tree()

< ROOT NODE > Node: 1 <  >
-------< A > Node: 3 < A B >
-------< B > Node: 4 <  >


In [16]:
print(chrest1.verbal.net)
print(chrest1.verbal.net.contents)
print(chrest1.verbal.net.children)
print(chrest1.verbal.net.children[0].child.contents)
print(chrest1.verbal.net.children[0].test)
print(chrest1.verbal.net.children[1].test)
print(chrest1.verbal.net.children[0].child.image)
print(chrest1.verbal.net.children[1].child.idx) #thus, Link has tlink and child=Node parts!

['ROOT NODE']
[<__main__.Link object at 0x000002A00F57EA20>, <__main__.Link object at 0x000002A00F59B438>]
['A']
['A']
['B']
['A', 'B']
4


In [56]:
chrest1.verbal.recognise_and_learn(list('AC$'))        #rerun few times to learn further
chrest1.verbal.print_tree()

< ROOT NODE > Node: 1 <  >
-------< A > Node: 3 < A B >
--------------< C > Node: 6 < A C >
-------< B > Node: 4 <  >
-------< C > Node: 5 <  >


In [64]:
print(chrest1.verbal.net.children[2].child.contents )
print(chrest1.verbal.net.children[0].child.children[0].child.image )
chrest1.clock

['C']
['A', 'C']


44000

1:17:33 1st ruby like

In [54]:
print(chrest1.__dict__)

{'nodes': [<__main__.Node object at 0x0000019CFE5E4D68>, <__main__.Node object at 0x0000019CFE8296A0>, <__main__.Node object at 0x0000019CFE7FE3C8>, <__main__.Node object at 0x0000019CFE7FEBA8>, <__main__.Node object at 0x0000019CFE8F5588>], 'visual': <__main__.Modality object at 0x0000019CFE8A3C18>, 'verbal': <__main__.Modality object at 0x0000019CFE7F49B0>, 'action': <__main__.Modality object at 0x0000019CFE7FE898>, 'modalities': {'visual': <__main__.Modality object at 0x0000019CFE8A3C18>, 'verbal': <__main__.Modality object at 0x0000019CFE7F49B0>, 'action': <__main__.Modality object at 0x0000019CFE7FE898>}, 'clock': 24000}


In [46]:
s1=[]
s2=[]
s1.append(s2)
s1

[[]]

In [25]:
chrest = Chrest()

inp = list(map(list, (
    'A$', 'AB$', 'ABC$'
)))

errors = inp
print(errors)
for i in range(5000):
    new_errors = []
    for pattern in errors:
        node = chrest.verbal.recognise_and_learn(pattern)
        if node.image != pattern:
            new_errors.append(pattern)
    errors = new_errors
    if len(errors) == 0:
        break
chrest.verbal.print_tree()
for pattern in inp:
    print("Recognises", ''.join(pattern), "as:", chrest.verbal.recognise(pattern).image)

[['A', '$'], ['A', 'B', '$'], ['A', 'B', 'C', '$']]
< ROOT NODE > Node: 1 <  >
-------< A > Node: 3 < A B C >
--------------< $ > Node: 7 < A >
--------------< B > Node: 8 < A B C >
---------------------< $ > Node: 9 < A B >
-------< B > Node: 4 <  >
-------< C > Node: 5 <  >
-------< $ > Node: 6 < $ >
Recognises A$ as: ['A']
Recognises AB$ as: ['A', 'B']
Recognises ABC$ as: ['A', 'B', 'C']


In [31]:
chrest = Chrest()

inp = list(map(list, (
    'A$', 'AB$', 'ABC$'
)))

errors = inp
for i in range(5000):
    new_errors = []
    for pattern in errors:
        node = chrest.verbal.recognise_and_learn(pattern)
        if node.image != pattern:
            new_errors.append(pattern)
    errors = new_errors
    if len(errors) == 0:
        break
chrest.verbal.print_tree()
for pattern in inp:
    print("Recognises", ''.join(pattern), "as:", chrest.verbal.recognise(pattern).image)

< ROOT NODE > Node: 1 <  >
-------< A > Node: 3 < A B C >
--------------< $ > Node: 7 < A >
--------------< B > Node: 8 < A B C >
---------------------< $ > Node: 9 < A B >
-------< B > Node: 4 <  >
-------< C > Node: 5 <  >
-------< $ > Node: 6 < $ >
Recognises A$ as: ['A']
Recognises AB$ as: ['A', 'B']
Recognises ABC$ as: ['A', 'B', 'C']


# Attention mech

In [32]:
def zliverpool_pure_attention():
    
    chrest = Chrest() #comment out to learn on top of previous knowledge, uncomment to train from blank slate
    inp = [list(map(list, x.split('~'))) for x in ['LIVERPOOL$~A$', 'MANCHESTER$~B$']]

    number_of_nodes =0

    #random.shuffle(inp) #uncomment to shuffle input patterns
    for i in range(5000): #set an arbitrary number of training cycles
        errors = 0        #set a stop condition for the training loop
        for pattern in inp:
            vis, verb = pattern
            time = chrest.clock   #check time before learning
            node1 = chrest.visual.recognise_and_learn(vis) #if learning is necessary, it will add to time
            node2 = chrest.verbal.recognise_and_learn(verb)
            
            if node1.idx>number_of_nodes: #node counter
                number_of_nodes = node1.idx
            
            if chrest.clock == time: #if no time spent on learning means the pattern is learnt -> create     naming link
                node1.label = node2.idx
                node2.label = node1.idx
            else:
                errors += 1
        if errors == 0:
            break

    chrest.visual.print_tree()
    chrest.verbal.print_tree()
    
        
    tests = list(map(list, (
         'ZZLIVERPOOL$','LIVERPOOZ$','LZVERPOOL$'
    )))
    window_size = 7 #attention window size
    try:
        for test in tests: #for each of the test patterns
            print()
            print('test_pattern:', ''.join(test))
            label = None
            for window_start in range(len(test)-2): #for each attention   window start points from 0 to length of pattern-2
                
                if window_start + window_size < len(test):   
#if the attention window size fits inside the pattern                 
                    window_end = window_start + window_size
                else:    #if the attention window size does not fit inside the pattern
                    window_end=None
                    
                attention_window = test[window_start:window_end] #
                print('Attention window:', ''.join(attention_window))

                node = chrest.visual.recognise(attention_window)
               
                if node.label: #if label is not None
                    label = node.label
                    print('recognises', ''.join(test),'as',''.join(node.image),', label:', chrest.nodes[label].image)
#                 else:
#                     print('recognises', ''.join(attention_window), 'as None!')
    except:
            
        print('None found!')
#     print(chrest.visual.recognise(list('LIVERPOOZ$')).image )
    print('number_of_nodes=', number_of_nodes)

zliverpool_pure_attention()


< ROOT NODE > Node: 0 <  >
-------< L > Node: 3 < L I V E R P O O L > (4)
-------< M > Node: 5 < M A N C H E S T E R > (6)
-------< I > Node: 7 <  >
-------< A > Node: 8 <  >
-------< V > Node: 9 <  >
-------< N > Node: 10 <  >
-------< E > Node: 11 <  >
-------< C > Node: 12 <  >
-------< R > Node: 13 <  >
-------< H > Node: 14 <  >
-------< P > Node: 15 <  >
-------< S > Node: 16 <  >
-------< O > Node: 17 <  >
-------< T > Node: 18 <  >
< ROOT NODE > Node: 1 <  >
-------< A > Node: 4 < A > (3)
-------< B > Node: 6 < B > (5)

test_pattern: ZZLIVERPOOL$
Attention window: ZZLIVER
Attention window: ZLIVERP
Attention window: LIVERPO
recognises ZZLIVERPOOL$ as LIVERPOOL , label: ['A']
Attention window: IVERPOO
Attention window: VERPOOL
Attention window: ERPOOL$
Attention window: RPOOL$
Attention window: POOL$
Attention window: OOL$
Attention window: OL$

test_pattern: LIVERPOOZ$
Attention window: LIVERPO
recognises LIVERPOOZ$ as LIVERPOOL , label: ['A']
Attention window: IVERPOO
Attention

# Big Chunk Zliverpool

In [33]:
def zliverpool_atten_big_chunk():
    
    chrest = Chrest() #comment out to learn on top of previous knowledge, uncomment to train from blank slate
    inp = [list(map(list, x.split('~'))) for x in ['LIVERPOOL$~A$', 'LIVE$~B$', 'LIVER$~C$']]

    number_of_nodes =0

    #random.shuffle(inp) #uncomment to shuffle input patterns
    for i in range(5000): #set an arbitrary number of training cycles
        errors = 0        #set a stop condition for he training loop
        for pattern in inp:
            vis, verb = pattern
            time = chrest.clock   #check time before learning
            node1 = chrest.visual.recognise_and_learn(vis) 
                           #if learning is necessary, it will add to time
            node2 = chrest.verbal.recognise_and_learn(verb)
            
            if node1.idx>number_of_nodes: #node counter
                number_of_nodes = node1.idx
            
            if chrest.clock == time: #if no time was spent on learning,   the pattern is learnt -> create      naming link
                node1.label = node2.idx
                node2.label = node1.idx
            else:
                errors += 1
        if errors == 0:
            break

    chrest.visual.print_tree()
    chrest.verbal.print_tree()
    
        
    tests = list(map(list, (
         'ZZLIVERZOOL$','LIVERPOOZ$'
    )))
    window_size = 7 #attention window size
    
    print('\n-------- MAX IMAGE METHOD --------\n') #i.e., the biggest     chunk activation method
    try:
        for test in tests: #for each of the test patterns
            print()
            print('test_pattern:', ''.join(test))
            label = None
            max_image = []  #intiialise the largest chunk activation measure
            for window_start in range(len(test)-2): 
#for each attention window start points (i.e., from 0 to the length_of_    pattern-2)
                
                if window_start + window_size < len(test):   
#if the attention window size fits inside the pattern                 
                    window_end = window_start + window_size
                else:                                        
#if the attention window size does not fit inside the pattern
                    window_end=None
                    
                attention_window = test[window_start:window_end] #
                print('Attention window:', ''.join(attention_window))

                node = chrest.visual.recognise(attention_window)
               
                if node.label: #if label is not None
                    if len(node.image) > len(max_image): 
#if the size of the current chunk is bigger than the largest chunk met so far
                        max_image = node.image   
#set the current chunk to be the largest chunk
                        label = node.label       
#take the naming link of the current chunk to be the label
            print('recognises', ''.join(test),'as',''.join(max_image),'label:', chrest.nodes[label].image)
#                 else:
#                     print('recognises', ''.join(attention_window), 'as None!')

    except:
            
        print('None found!')
#     print(chrest.visual.recognise(list('LIVERPOOZ$')).image )
    print('number_of_nodes=', number_of_nodes)

zliverpool_atten_big_chunk()


< ROOT NODE > Node: 0 <  >
-------< L > Node: 3 < L I V E R > {18}
--------------< I > Node: 11 < L I V E R > (5)
---------------------< V > Node: 13 < L I V E R >
----------------------------< E > Node: 14 < L I V E R P > {16}
-----------------------------------< R > Node: 16 < L I V E R P O O L > (4) {18, 14}
------------------------------------------< $ > Node: 18 < L I V E R > (7) {16, 3}
-----------------------------------< $ > Node: 17 < L I V E > (5)
-------< I > Node: 6 <  >
-------< V > Node: 8 <  >
-------< E > Node: 9 <  >
-------< R > Node: 10 <  >
-------< P > Node: 12 <  >
-------< $ > Node: 15 < $ > (5)
-------< O > Node: 19 <  >
< ROOT NODE > Node: 1 <  >
-------< A > Node: 4 < A > (16)
-------< B > Node: 5 < B > (17)
-------< C > Node: 7 < C > (18)

-------- MAX IMAGE METHOD --------


test_pattern: ZZLIVERZOOL$
Attention window: ZZLIVER
Attention window: ZLIVERZ
Attention window: LIVERZO
Attention window: IVERZOO
Attention window: VERZOOL
Attention window: ERZOOL$
Att

In [34]:
a = [1,2,3,4]

In [35]:
a[1:None]

[2, 3, 4]

In [40]:
y= [num for num in range(5)]

In [41]:
y

[0, 1, 2, 3, 4]